<a href="https://colab.research.google.com/github/akbarehsani/backsample2/blob/master/multi_processing_NN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import argparse

import multiprocessing as mp
import sys
import threading
import time
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision


def model_loop_fn(index, args):
  print('Thread {} started'.format(index))
  start_time = time.time()
  data = torch.randn(args.batch_size, 3, 224, 224)
  target = torch.zeros(args.batch_size, dtype=torch.int64)
  model = torchvision.models.resnet18()
  loss_fn = nn.NLLLoss()
  optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.5)
  for i in range(0, args.loops):
    optimizer.zero_grad()
    output = model(data)
    loss = loss_fn(output, target)
    loss.backward()
    optimizer.step()
  print('Thread {} took {:.3f}ms'.format(index,
                                         1000.0 * (time.time() - start_time)))


def hog_loop_fn(index, args):
  print('Thread {} started'.format(index))
  start_time = time.time()
  for i in range(0, args.loops):
    x = args.hog_count
    while x > 0:
      x -= 1
  print('Thread {} took {:.3f}ms'.format(index,
                                         1000.0 * (time.time() - start_time)))


def bench_resnet(args):
  loop_fn = hog_loop_fn if args.hog else model_loop_fn
  threads = []
  for i in range(0, args.num_threads):
    if args.mp:
      thread = mp.Process(target=loop_fn, args=(i, args,))
    else:
      thread = threading.Thread(target=loop_fn, args=(i, args,))
    thread.daemon = True
    threads.append(thread)
    thread.start()
  for thread in threads:
    thread.join()


if __name__ == '__main__':
  parser = argparse.ArgumentParser(add_help=False)
  parser.add_argument('--batch_size', type=int, default=4)
  parser.add_argument('--loops', type=int, default=10)
  parser.add_argument('--num_threads', type=int, default=1)
  parser.add_argument('--mp', action='store_true')
  parser.add_argument('--hog', action='store_true')
  parser.add_argument('--hog_count', type=int, default=10000000)
  args = parser.parse_args()

  bench_resnet(args)

usage: ipykernel_launcher.py [--batch_size BATCH_SIZE] [--loops LOOPS]
                             [--num_threads NUM_THREADS] [--mp] [--hog]
                             [--hog_count HOG_COUNT]
ipykernel_launcher.py: error: unrecognized arguments: -f /root/.local/share/jupyter/runtime/kernel-d59223ba-8c51-40ae-b5f3-1b5c8f5b9448.json


SystemExit: ignored

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:3334: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
